# Dominance & Cycles in Chip-Clearing Strategies (variable m, exact computation)

This notebook generates **all strategies** (chip allocations) for given hit probabilities `p` and total chip count `total_chips`, compares every strategy against every other **exactly** (using `Fraction`), and analyzes the **dominance relation** as well as **non-transitive 3-cycles**.

---

## Inputs (at the top of the code)

```python
p = (Fraction(1,2), Fraction(1,3), Fraction(1,6))  # hit probabilities
total_chips = 6                                    # total chips per strategy
```

## What is computed

1. **All strategies**  
   All m-tuples of nonnegative integers summing to `total_chips`.

2. **Exact pairwise comparisons**  
   For each pair (A, B), $(P_A, P_B, P_U)$ is computed exactly.  
   - `beats[A]` = set of strategies that A **beats** (`P_A > P_B`)  
   - `loses[A]` = opponents that beat A (`P_B > P_A`)  
   - `ties[A]`  = **ties** (`P_A = P_B`)

3. **Top-5 strategies (ranking)**  
   Sorted by **net dominance** `|beats| − |loses|`  
   (tie-breaker: `|beats|`).

4. **Dominant strategies**  
   A strategy is *dominant* if it **wins against all others**,  
   i.e. `len(loses[S]) == 0` and `len(beats[S]) > 0`.

5. **Non-transitive 3-cycles**  
   Triples $(S,T,W)$ with  
   $S \succ T$, $T \succ W$, $W \succ S$.  
   Duplicate cycles (rotations) are removed using a  
   **canonical representation**.

6. **Example computation**  
   For the **first** detected 3-cycle, the exact probabilities of the three
   pairwise matchups are printed.


## Console output (interpretation)

- **Top-5 strategies**:  
  `beats`, `loses`, `ties` give the out-degrees, in-degrees, and ties in the
  dominance graph. A higher net value ⇒ “stronger” in direct comparison.

- **Dominant strategy/strategies**:  
  If any exist, they are listed explicitly.

- **Total number of 3-cycles**:  
  All detected cycles, **without** duplicates (rotations).  
  Directly below: **example output** (at most the first 5).  
  → You can control the number of displayed examples via `cycles[:n]`.

- **Example computation for a cycle**:  
  Exact fractions and values rounded to three decimals for $P_A, P_B, P_U$ in the three matchups of the cycle.

---

In [5]:
from fractions import Fraction
from functools import lru_cache
from itertools import combinations, permutations

# ============================================================
# Parameters – input
# ============================================================

p = (Fraction(1,2), Fraction(1,3), Fraction(1,6))   # hit probabilities (m fields)
total_chips = 5                                     # total chips per strategy

# ============================================================
# Exact single-game probabilities (general m)
# ============================================================

def exact_probabilities_fraction(p, A, B):
    p = tuple(Fraction(pi) for pi in p)
    A = tuple(A); B = tuple(B)

    @lru_cache(None)
    def P_A(V, W):
        sum_V, sum_W = sum(V), sum(W)
        if sum_V == 0 and sum_W == 0: return Fraction(0, 1)  # simultaneous end → draw → contribution 0 for A
        if sum_V == 0: return Fraction(1, 1)                  # A finished, B not → A wins.
        if sum_W == 0: return Fraction(0, 1)                  # B finished, A not → A loses

       # "Relevant" fields (at least one chip remains somewhere)
        s = sum(pj for pj, vj, wj in zip(p, V, W) if vj or wj)
        if s == 0: return Fraction(0, 1)

        acc = Fraction(0, 1)
        for j, pj in enumerate(p):
            if V[j] or W[j]:
                Vn = V[:j] + (max(0, V[j]-1),) + V[j+1:]
                Wn = W[:j] + (max(0, W[j]-1),) + W[j+1:]
                acc += (pj / s) * P_A(Vn, Wn)
        return acc

    PA = P_A(A, B)
    PB = P_A(B, A)
    PU = Fraction(1, 1) - PA - PB
    return PA, PB, PU

# ============================================================
# All strategies (compositions) for m fields, sum = total_chips
# ============================================================

def all_strategies(total, m):
   # recursive generation: all m-tuples of nonnegative integers with sum = total

    if m == 1:
        yield (total,)
        return
    for x in range(total + 1):
        for rest in all_strategies(total - x, m - 1):
            yield (x,) + rest

m = len(p)
strategies = list(all_strategies(total_chips, m))

print(f"p = {p}  |  m = {m}  |  total_chips = {total_chips}")
print(f"Number of strategies: {len(strategies)}  (≈ C(m+N-1, m-1))")

# ============================================================
# Pairwise comparisons
# ============================================================

def compare(A, B):
    PA, PB, PU = exact_probabilities_fraction(p, A, B)
    if PA > PB: return 1, PA, PB, PU  # A dominiert B
    if PB > PA: return -1, PA, PB, PU # B dominiert A
    return 0, PA, PB, PU              # Remis

beats = {S: set() for S in strategies}
loses = {S: set() for S in strategies}
ties  = {S: set() for S in strategies}
probs = {}  # (A,B) -> (PA,PB,PU)

for A, B in combinations(strategies, 2):
    sgn, PA, PB, PU = compare(A, B)
    probs[(A,B)] = (PA, PB, PU)
    probs[(B,A)] = (PB, PA, PU)
    if sgn == 1:
        beats[A].add(B); loses[B].add(A)
    elif sgn == -1:
        beats[B].add(A); loses[A].add(B)
    else:
        ties[A].add(B); ties[B].add(A)

# ============================================================
# Metrics & Top-5
# ============================================================

# Net dominance score: (beats - loses), tie-breaker: (beats)

def score(S):
    return (len(beats[S]) - len(loses[S]), len(beats[S]))

top5 = sorted(strategies, key=score, reverse=True)[:5]
dominant = [S for S in strategies if len(loses[S]) == 0 and len(beats[S]) > 0]

print("\nTop-5 strategies (by net dominance, then 'beats'):")
for rank, S in enumerate(top5, 1):
    print(f"{rank:2d}. {S}   | beats={len(beats[S])}, loses={len(loses[S])}, ties={len(ties[S])}")

print("\nDominant strategy/strategies:", dominant if dominant else "none")

# ============================================================
# Non-transitive 3-cycles (S > T > W > S), up to 5 examples
# Efficient search: follow edges (S→T) and test candidates W in beats[T] with W→S
# ============================================================

def canonical_cycle(triple):
    # rotiere, so dass der lexikographisch kleinste Start vorne steht (Richtung fest)
    S,T,W = triple
    rots = [(S,T,W), (T,W,S), (W,S,T)]
    return min(rots)


# ---------- Non-transitive 3-cycles (S > T > W > S), without rotational duplicates ----------
cycles_set = set()
for S, T, W in permutations(strategies, 3):
    if T in beats[S] and W in beats[T] and S in beats[W]:
        cycles_set.add(canonical_cycle((S, T, W)))

cycles = sorted(list(cycles_set))

# ============================================================
# Output
# ============================================================

print(f"\nTotal number of 3-cycles (S ≻ T ≻ W ≻ S): {len(cycles)}")
print()
print("Example output (max. 5):")
for cyc in cycles[:5]:
    print(cyc)

# Optional: example probabilities for the first cycle
print()
print("Example computation for one cycle")
if cycles:
    S, T, W = cycles[0]
    for A, B in [(S, T), (T, W), (W, S)]:
        PA, PB, PU = probs[(A, B)]
        print(f"\n{A} vs {B}:  PA={PA}  PB={PB}  PU={PU}")
        print(f"\n{A} vs {B}:  PA={float(PA):.3f}  PB={float(PB):.3f}  PU={float(PU):.3f}")
        print()


p = (Fraction(1, 2), Fraction(1, 3), Fraction(1, 6))  |  m = 3  |  total_chips = 5
Number of strategies: 21  (≈ C(m+N-1, m-1))

Top-5 strategies (by net dominance, then 'beats'):
 1. (2, 2, 1)   | beats=19, loses=1, ties=0
 2. (3, 1, 1)   | beats=19, loses=1, ties=0
 3. (3, 2, 0)   | beats=18, loses=2, ties=0
 4. (4, 1, 0)   | beats=18, loses=2, ties=0
 5. (2, 3, 0)   | beats=16, loses=4, ties=0

Dominant strategy/strategies: none

Total number of 3-cycles (S ≻ T ≻ W ≻ S): 6

Example output (max. 5):
((0, 3, 2), (0, 4, 1), (3, 0, 2))
((0, 3, 2), (1, 4, 0), (3, 0, 2))
((0, 4, 1), (3, 0, 2), (1, 2, 2))
((1, 2, 2), (1, 4, 0), (3, 0, 2))
((2, 2, 1), (3, 1, 1), (4, 1, 0))

Example computation for one cycle

(0, 3, 2) vs (0, 4, 1):  PA=131/243  PB=112/243  PU=0

(0, 3, 2) vs (0, 4, 1):  PA=0.539  PB=0.461  PU=0.000


(0, 4, 1) vs (3, 0, 2):  PA=961/1875  PB=914/1875  PU=0

(0, 4, 1) vs (3, 0, 2):  PA=0.513  PB=0.487  PU=0.000


(3, 0, 2) vs (0, 3, 2):  PA=114739/337500  PB=919943/7200000  PU